## Deliverable 2. Create a Customer Travel Destinations Map.

In [183]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Add Matplotlib inline magic command
%matplotlib inline
import matplotlib as plt
# Import API key
from config import g_key
import matplotlib.pyplot as plt
import numpy as np
import seaborn
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [204]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,WeatherPy_Database,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.12,-134.97,72.48,78,47,11.12,scattered clouds
1,1,Yumen,CN,40.28,97.20,86.54,12,98,23.71,overcast clouds
2,2,Winnemucca,US,40.97,-117.74,53.60,18,1,6.93,clear sky
3,3,Qaanaaq,GL,77.48,-69.36,37.31,87,100,16.96,light rain
4,4,Tasiilaq,GL,65.61,-37.64,48.20,60,69,5.82,broken clouds


In [205]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 40
What is the maximum temperature you would like for your trip? 110


In [206]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,WeatherPy_Database,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.12,-134.97,72.48,78,47,11.12,scattered clouds
1,1,Yumen,CN,40.28,97.20,86.54,12,98,23.71,overcast clouds
2,2,Winnemucca,US,40.97,-117.74,53.60,18,1,6.93,clear sky
4,4,Tasiilaq,GL,65.61,-37.64,48.20,60,69,5.82,broken clouds
5,5,Sangar,RU,63.92,127.47,65.59,52,0,7.45,clear sky


In [207]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


WeatherPy_Database     0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [208]:
preferred_cities_df.count()

WeatherPy_Database     557
City                   557
Country                553
Lat                    557
Lng                    557
Max Temp               557
Humidity               557
Cloudiness             557
Wind Speed             557
Current Description    557
dtype: int64

In [209]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna(how='any')

In [210]:
preferred_cities_df.isnull().sum()

WeatherPy_Database     0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [211]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng
0,Rikitea,PF,72.48,scattered clouds,-23.12,-134.97
1,Yumen,CN,86.54,overcast clouds,40.28,97.20
2,Winnemucca,US,53.60,clear sky,40.97,-117.74
4,Tasiilaq,GL,48.20,broken clouds,65.61,-37.64
5,Sangar,RU,65.59,clear sky,63.92,127.47
...,...,...,...,...,...,...
565,Kloulklubed,PW,82.40,broken clouds,7.04,134.26
566,Port Macquarie,AU,51.01,few clouds,-31.43,152.92
567,Kapuskasing,CA,50.00,mist,49.42,-82.43
568,Sinnamary,GF,77.67,light rain,5.38,-52.95


In [212]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
dtype: int64

In [213]:
# 5b. Create a new column "Hotel Name"
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,72.48,scattered clouds,-23.12,-134.97,
1,Yumen,CN,86.54,overcast clouds,40.28,97.20,
2,Winnemucca,US,53.60,clear sky,40.97,-117.74,
4,Tasiilaq,GL,48.20,broken clouds,65.61,-37.64,
5,Sangar,RU,65.59,clear sky,63.92,127.47,
6,Butaritari,KI,82.60,scattered clouds,3.07,172.79,
7,Cayenne,GF,75.20,light rain,4.93,-52.33,
8,Itoman,JP,84.20,broken clouds,26.12,127.67,
9,Kindu,CD,99.39,few clouds,-2.95,25.95,
10,Provideniya,RU,43.41,broken clouds,64.38,-173.30,


In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
}
# Use base URL to search for hotels in Paris.
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(50)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [ ]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "Weather_Database/City_ID.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
WeatherPy_vacation_map = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
WeatherPy_vacation_map.add_layer(marker_layer)
plt.savefig("Weather_Database/WeatherPy_vacation_map.png")
# 11b. Display the figure
fig
